In [1]:
import numpy as np
import pandas as pd

import binance
from poloniex import Poloniex

import sched, time

import datetime

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


binance.set("api_key", "api_secret")

polo = Poloniex()

s = sched.scheduler(time.time, time.sleep)


def SendNotif(comp1, comp2):
    if (comp1 == True or comp2 == True):
        fromaddr = "email@address.com"
        toaddr = "email@address.com"
        msg = MIMEMultipart()
        msg['From'] = fromaddr
        msg['To'] = toaddr
        msg['Subject'] = "Notification for crypto LTC"
        
        if (comp1 == True and comp2 == True):
            body = "Bid (Binance) > Ask (Poloniex) AND Bid (Poloniex) > Ask (Binance)"
        elif (comp1 == True):
            body = "Bid (Binance) > Ask (Poloniex)"
        else:
            body = "Bid (Poloniex) > Ask (Binance)"
            
        msg.attach(MIMEText(body, 'plain'))
 
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(fromaddr, "password")
        text = msg.as_string()
        server.sendmail(fromaddr, toaddr, text)
        server.quit()


def TradingBot(sc): 

    print ("Analyzing the market...")
    print('Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
    print()
    
    #-------------------------------------------------------
    #Retrieve the asset list of an exchange A
    print("---------------------------------------------------------------------------")
    print()
    print("Retrieving asset list from Binance")
    pricesA = binance.tickers()
    ExA = pd.DataFrame(pricesA).transpose()
    print(ExA.head(5))  
    print("...")
    #-------------------------------------------------------
    #Retrieve the asset list of an exchange B
    print("---------------------------------------------------------------------------")
    print()
    print("Retrieving asset list from Poloniex")
    pricesB = dict(polo.returnTicker())
    ExB = pd.DataFrame.from_dict(pricesB, orient = 'index')
    print(ExB.head(5))
    print("...")
    #-------------------------------------------------------
    
    #-------------------------------------------------------
    #Retrieve the bid and ask price for a select currency
    print("---------------------------------------------------------------------------")
    print()
    print("Retrieving bid and ask price for LTC from Binance")
    LTC_A = ExA.loc['LTCBTC']
    print(LTC_A)
    print()
    
    print("---------------------------------------------------------------------------")
    print()
    print("Retrieving bid and ask price for LTC from Binance")
    LTC_B = ExB.loc['BTC_LTC']
    print(LTC_B)
    print()
    #-------------------------------------------------------
    
    #-------------------------------------------------------
    #Compare bidA/askB and bidB/askA
    print("---------------------------------------------------------------------------")
    print()
    print("Exchange A : Binance")
    print("Exchange B : Poloniex")
    print()
    print("Comparing bidA/askB and bidB/askA")
    print()
    A = (float(LTC_A['bid']), float(LTC_A['ask']))
    B = (LTC_B['highestBid'], LTC_B['lowestAsk'])
    
    A = np.array(A)
    B = np.array(B)
    
    Comp = pd.DataFrame([A,B], index = 'A B'.split(), columns = 'Bid Ask'.split())
    print(Comp)
    
    Comp1 = Comp['Bid']['A'] > Comp['Ask']['B']
    Comp2 = Comp['Bid']['B'] > Comp['Ask']['A']
    
    print()
    print("Bid A > Ask B : {}".format(Comp1))
    print("Bid B > Ask A : {}".format(Comp2))
    print()
    #-------------------------------------------------------
    
    #-------------------------------------------------------
    #Send Email Notification
    SendNotif(Comp1, Comp2)
    #-------------------------------------------------------
    
    s.enter(60, 1, TradingBot, (sc,))


s.enter(60, 1, TradingBot, (s,))
s.run()

Analyzing the market...
Timestamp: 2020-09-13 18:01:30

---------------------------------------------------------------------------

Retrieving asset list from Binance
                bid         ask        bidQty        askQty
ETHBTC   0.03525900  0.03526200    2.03700000   22.13800000
LTCBTC   0.00465300  0.00465500   88.16000000    2.79000000
BNBBTC   0.00300510  0.00300610    2.18000000    0.04000000
NEOBTC   0.00191200  0.00191300   37.96000000   29.04000000
QTUMETH  0.00708900  0.00712400  288.84000000  288.90000000
...
---------------------------------------------------------------------------

Retrieving asset list from Poloniex
          baseVolume      high24hr    highestBid  id  isFrozen          last  \
BTC_BTS     3.130312  3.170000e-06  2.760000e-06  14         0  2.750000e-06   
BTC_DASH    4.535914  7.593720e-03  7.210430e-03  24         0  7.217990e-03   
BTC_DOGE    1.940143  2.800000e-07  2.600000e-07  27         0  2.600000e-07   
BTC_LTC    18.896271  4.889350e-03 

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))